#### removing points that are less than 30m from each other
and share classification as val/training and are same location-date

In [7]:

import pandas as pd

# Load the file of points that are less than 30m apart from another point on the same date/location
# does also include clusters of more than two points
df = pd.read_csv("data/AllTraining_080425prj_closepts.csv")

# Set the threshold distance (m)
threshold = 30

df

,Location,Date,Type,Region,Latitude,Longitude,Split,Date_Loc,NEAR_FID,NEAR_DIST,Orig_FID
0,Oxnard,11/9/2023,hoop,NaN,34.241722,-119.132067,1,2023-11-09 Oxnard,254,26.160596,226
1,Oxnard,5/6/2023,hoop,NaN,34.151197,-119.104755,1,2023-05-06 Oxnard,320,28.005998,235
2,Oxnard,10/14/2019,hoop,NaN,34.189632,-119.123383,2,2019-10-14 Oxnard,305,28.300004,243
3,Oxnard,10/14/2019,hoop,NaN,34.189403,-119.123544,2,2019-10-14 Oxnard,243,29.336011,245
4,Oxnard,11/9/2023,hoop,NaN,34.232067,-119.136343,1,2023-11-09 Oxnard,281,29.844050,249
...,...,...,...,...,...,...,...,...,...,...,...
1713,Santa Maria,2/6/2021,other,NaN,35.046120,-120.574415,2,2021-02-06 Santa Maria,5782,27.970748,5783
1714,Santa Maria,2/6/2021,other,NaN,35.042122,-120.577924,2,2021-02-06 Santa Maria,5795,28.209781,5793
1715,Santa Maria,2/6/2021,other,NaN,35.042084,-120.577618,2,2021-02-06 Santa Maria,5793,28.209781,5795
1716,Santa Maria,2/6/2021,mulch,NaN,34.998795,-120.502767,2,2021-02-06 Santa Maria,5874,27.924171,5870


In [ ]:

# Track pairs and points to delete
delete_ids = set()
processed_pairs = set()

# Iterate through rows
for _, row in df.iterrows():
    oid = row['Orig_FID']
    near_fid = row['NEAR_FID']
    near_dist = row['NEAR_DIST']

    if pd.notnull(near_fid) and near_dist < threshold:
        pair = tuple(sorted([int(oid), int(near_fid)]))
        # If either oid or near_fid is already marked for deletion, skip this pair
        if int(oid) in delete_ids or int(near_fid) in delete_ids:
            pass
        else:
            # Check if oid or near_fid appears elsewhere in NEAR_FID column (excluding this row)
            oid_in_near_fid = ((df['NEAR_FID'] == int(oid)) & (df.index != row.name)).any()
            near_fid_in_near_fid = ((df['NEAR_FID'] == int(near_fid)) & (df.index != row.name)).any()
            # if oid appears in NEAR_FID but near_fid does not, delete oid
            if oid_in_near_fid and not near_fid_in_near_fid:
                delete_ids.add(int(oid))
            # if near_fid appears in NEAR_FID but oid does not, delete near_fid
            elif near_fid_in_near_fid and not oid_in_near_fid:
                delete_ids.add(int(near_fid))
            else:
                # If both or neither appears elsewhere, delete the point with higher OID
                delete_ids.add(max(pair))
        processed_pairs.add(pair) 


# Filter out the points to delete
filtered_df = df[~df['Orig_FID'].isin(delete_ids)]

# Save the cleaned data
# filtered_df.to_csv("data/AllTraining_080425prj_FilteredPts.csv")

print(f"Deleted {len(delete_ids)} points within {threshold} meters of another point.")
filtered_df


# save the points that were deleted as a csv
deleted_df = df[df['Orig_FID'].isin(delete_ids)]
# deleted_df.to_csv("data/AllTraining_080425prj_DeletedPts.csv")


Deleted 744 points within 30 meters of another point.
